# Data Source

Link: https://www.kaggle.com/c/instacart-market-basket-analysis/data

# Import Modules

In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

# Import CSV files

In [122]:
orders_original = pd.read_csv('orders.csv')

In [2]:
aisles = pd.read_csv('aisles.csv')

In [3]:
departments = pd.read_csv('departments.csv')

In [4]:
orders = pd.read_csv('orders.csv')

In [5]:
products = pd.read_csv('products.csv')

In [6]:
o_p_prior = pd.read_csv('order_products__prior.csv')

Find further information within CSV files.

In [7]:
aisles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


In [8]:
aisles.head(3)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars


In [9]:
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 464.0+ bytes


In [10]:
departments.head(3)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery


In [11]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [12]:
orders.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


In [13]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [14]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [15]:
o_p_prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [16]:
o_p_prior.head(3)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0


# Focus on 'Prior' Data

In [ ]:
orders_prior_original = orders_original.loc[orders_original.eval_set == 'prior']

In [17]:
orders_prior = orders.loc[orders.eval_set == 'prior']

I am keeping one dataframe to use for joining with other dataframes and the original with just filtering for 'prior' and leaving alone. So, the original orders dataframe will not have the products listed in the orders.

In [18]:
orders_prior.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3214874 entries, 0 to 3421081
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 196.2+ MB


Merge Department & Aisle into Products

In [19]:
p_a = products.join(aisles.set_index('aisle_id'), on='aisle_id')

In [20]:
p_a.head(3)

,product_id,product_name,aisle_id,department_id,aisle
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes
1,2,All-Seasons Salt,104,13,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea


In [21]:
p_a_d = p_a.join(departments.set_index('department_id'), on='department_id')
p_a_d.head(3)

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages


In [22]:
p_a_d = p_a_d[['product_id','product_name', 'aisle', 'department']]
p_a_d.head(3)

,product_id,product_name,aisle,department
0,1,Chocolate Sandwich Cookies,cookies cakes,snacks
1,2,All-Seasons Salt,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,tea,beverages


In [23]:
p_a_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    49688 non-null  int64 
 1   product_name  49688 non-null  object
 2   aisle         49688 non-null  object
 3   department    49688 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


Merge Products + Aisle + Departments into Prior Orders so we now have the products, aisle, and department for each order

In [24]:
o_p_a_d = o_p_prior.join(p_a_d.set_index('product_id'), on='product_id')
o_p_a_d.head(3)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle,department
0,2,33120,1,1,Organic Egg Whites,eggs,dairy eggs
1,2,28985,2,1,Michigan Organic Kale,fresh vegetables,produce
2,2,9327,3,0,Garlic Powder,spices seasonings,pantry


In [25]:
o_p_a_d = o_p_a_d[['order_id','product_id','product_name', 'aisle', 'department', 'add_to_cart_order','reordered']]
o_p_a_d.head(3)

,order_id,product_id,product_name,aisle,department,add_to_cart_order,reordered
0,2,33120,Organic Egg Whites,eggs,dairy eggs,1,1
1,2,28985,Michigan Organic Kale,fresh vegetables,produce,2,1
2,2,9327,Garlic Powder,spices seasonings,pantry,3,0


In [26]:
orders_final = orders_prior.join(o_p_a_d.set_index('order_id'), on='order_id')
orders_final.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,product_name,aisle,department,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,Soda,soft drinks,beverages,1,0
0,2539329,1,prior,1,2,8,NaN,14084,Organic Unsweetened Vanilla Almond Milk,soy lactosefree,dairy eggs,2,0
0,2539329,1,prior,1,2,8,NaN,12427,Original Beef Jerky,popcorn jerky,snacks,3,0


In [27]:
orders_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 3421081
Data columns (total 13 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
 7   product_id              int64  
 8   product_name            object 
 9   aisle                   object 
 10  department              object 
 11  add_to_cart_order       int64  
 12  reordered               int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 3.4+ GB


In [28]:
orders_final = orders_final[['order_id', 'user_id','order_number','order_dow','order_hour_of_day','days_since_prior_order','product_id','product_name', 'aisle', 'department', 'add_to_cart_order','reordered']]

In [29]:
orders_final.head(3)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,product_name,aisle,department,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,196,Soda,soft drinks,beverages,1,0
0,2539329,1,1,2,8,NaN,14084,Organic Unsweetened Vanilla Almond Milk,soy lactosefree,dairy eggs,2,0
0,2539329,1,1,2,8,NaN,12427,Original Beef Jerky,popcorn jerky,snacks,3,0


# Find which users have more than 1 order (are returning users)

Creating a groupby based on user_id and find the maximum number of their order number. If the user is not a returning customer, then their maximum order number will be 1. If they are, then it will be greater than 1.

In [137]:
users = orders_original.groupby('user_id')['order_number'].max()

In [138]:
users.sort_values()

user_id
127633     3
74147      3
160570     3
160560     3
74171      3
          ..
1868      99
31118     99
122476    99
120897    99
26086     99
Name: order_number, Length: 206209, dtype: int64

The minimum maximum order number is 3, meaning all users have made at least 3 orders.

Turns out all the customers are returning customers!

# Export to CSV

In [46]:
#orders_final.to_csv('orders_prior_final.csv', index=False)

# T-Test

Null Hypothesis: the average number of days since the prior order from their second order is the same as their third order.

In [132]:
second_order = orders_original.loc[orders_original.order_number == 2]
third_order = orders_original.loc[orders_original.order_number == 3]

In [133]:
second_order_mean = second_order.days_since_prior_order.mean()
second_order_mean

15.342128617082668

In [134]:
third_order_mean = third_order.days_since_prior_order.mean()
third_order_mean

15.287014630787212

In [135]:
stats.ttest_ind(second_order['days_since_prior_order'], third_order['days_since_prior_order'], equal_var=False)

Ttest_indResult(statistic=1.7226585829918404, pvalue=0.08495103390895405)

According to the p-value not being less than 0.05, we fail to reject the null hypothesis. Meaning: based on the sample data, we do not have enough to reject the hypothesis that the average number of days since the prior order from their second order as their third order.